In [ ]:
#Prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ==========================================================
# 📂 Paths and Setup
# ==========================================================
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "prophet_state_all_output"
output_csv = os.path.join(output_folder, "prophet_all_predictions")
output_graphs = os.path.join(output_folder, "prophet_all_graphs")
future_forecast_csv = os.path.join(output_folder, "prophet_future_forecasts")
metrics_file = os.path.join(output_folder, "prophet_all_metrics.csv")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(future_forecast_csv, exist_ok=True)

# ==========================================================
# 🧮 Helper Functions
# ==========================================================
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

# ==========================================================
# 📘 Load Dataset
# ==========================================================
print(f"📘 Loading dataset: {input_file}")
df = pd.read_excel(input_file)
df.columns = [col.strip().lower() for col in df.columns]

required_cols = ['date', 'district', 'crop_type', 'average']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"❌ Missing required columns: {required_cols}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])

# ==========================================================
# 📊 Metrics Storage
# ==========================================================
metrics_list = []

# ==========================================================
# 🔁 Prophet per (District × Crop)
# ==========================================================
groups = df.groupby(['district', 'crop_type'])

for (district, crop), group in groups:
    print(f"\n🚀 Processing: District = {district} | Crop = {crop}")

    data = group.copy().sort_values('date')
    data = data.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')

    # Fill missing values
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # ✅ If too few data points, replace with crop-level average across districts
    if data['average'].count() < 2:
        print(f"⚠️ {district} - {crop}: Not enough data. Using crop-level average instead.")
        crop_level = df[df['crop_type'] == crop].groupby('date', as_index=False)['average'].mean()
        if crop_level.empty:
            print(f"⚠️ {district} - {crop}: No crop-level data found. Using overall average instead.")
            crop_level = df.groupby('date', as_index=False)['average'].mean()
        crop_level.set_index('date', inplace=True)
        crop_level = crop_level.asfreq('D')
        crop_level['average'] = crop_level['average'].ffill().bfill().fillna(crop_level['average'].mean())
        data = crop_level.copy()

    # ==========================================================
    # 🧩 Feature Engineering
    # ==========================================================
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(window=7).mean()
    data['MA_30'] = data['average'].rolling(window=30).mean()
    data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']] = data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].bfill().ffill()

    # ==========================================================
    # 🤖 Prophet Model Preparation
    # ==========================================================
    prophet_df = data.reset_index()[['date', 'average']].rename(columns={'date': 'ds', 'average': 'y'})

    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='additive'
    )

    try:
        model.fit(prophet_df)
    except Exception as e:
        print(f"❌ Skipping {district} - {crop} due to Prophet fit error: {e}")
        continue

    # ==========================================================
    # 📅 Forecast (Historical + Future)
    # ==========================================================
    forecast = model.predict(model.make_future_dataframe(periods=0, freq='D'))

    merged = prophet_df.merge(forecast[['ds', 'yhat']], on='ds', how='left')
    merged.rename(columns={'y': 'Actual', 'yhat': 'Predicted'}, inplace=True)
    merged['District'] = district
    merged['Crop'] = crop
    merged[['Actual', 'Predicted']] = merged[['Actual', 'Predicted']].round(2)

    # ==========================================================
    # 📈 Metrics Calculation
    # ==========================================================
    y_true = merged['Actual'].values
    y_pred = merged['Predicted'].values
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'District': district,
        'Crop': crop,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # ==========================================================
    # 💾 Save Historical Predictions
    # ==========================================================
    merged.to_csv(os.path.join(output_csv, f"{district}_{crop}_prophet_daily.csv"), index=False, float_format='%.2f')

    # ==========================================================
    # 🔮 Generate Future Forecast (30 Days)
    # ==========================================================
    future_periods = 30
    future_df = model.make_future_dataframe(periods=future_periods, freq='D')
    future_forecast = model.predict(future_df)

    future_forecast = future_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(future_periods)
    future_forecast[['yhat', 'yhat_lower', 'yhat_upper']] = future_forecast[['yhat', 'yhat_lower', 'yhat_upper']].round(2)
    future_forecast['District'] = district
    future_forecast['Crop'] = crop

    future_forecast.to_csv(os.path.join(future_forecast_csv, f"{district}_{crop}_future_forecast.csv"), index=False)

    # ==========================================================
    # 📊 Save Graph (Historical + Future)
    # ==========================================================
    plt.figure(figsize=(12,6))
    plt.plot(merged['ds'], merged['Actual'], label='Actual', color='blue')
    plt.plot(merged['ds'], merged['Predicted'], label='Predicted (Prophet)', color='red', linestyle='dashed')
    plt.plot(future_forecast['ds'], future_forecast['yhat'], label='Future Forecast (30 days)', color='green', linestyle='dotted')
    plt.title(f"Prophet Forecast - {district} | {crop}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{district}_{crop}_prophet_graph.png"))
    plt.close()

    del data, prophet_df, merged, model, forecast
    gc.collect()

# ==========================================================
# 🧾 Save Metrics Summary
# ==========================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All combinations processed successfully!")
print(f"✅ Metrics saved to: {metrics_file}")
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx

🚀 Processing: District = All Districts | Crop = full_state


00:18:54 - cmdstanpy - INFO - Chain [1] start processing
00:19:02 - cmdstanpy - INFO - Chain [1] done processing


✅ All Districts - full_state | MAE=148.73, RMSE=199.35, R²=0.8766, MAPE=7.89%, Accuracy=92.11%

🚀 Processing: District = Bagalkot | Crop = onion


00:19:07 - cmdstanpy - INFO - Chain [1] start processing
00:19:13 - cmdstanpy - INFO - Chain [1] done processing


✅ Bagalkot - onion | MAE=378.66, RMSE=550.26, R²=0.4212, MAPE=36.42%, Accuracy=63.58%

🚀 Processing: District = Bagalkot | Crop = wheat


...

✅ Udupi - tomato | MAE=697.79, RMSE=1052.64, R²=0.3508, MAPE=42.36%, Accuracy=57.64%


00:29:57 - cmdstanpy - INFO - Chain [1] start processing



🚀 Processing: District = Udupi | Crop = wheat


00:29:58 - cmdstanpy - INFO - Chain [1] done processing


✅ Udupi - wheat | MAE=0.37, RMSE=0.84, R²=1.0, MAPE=0.02%, Accuracy=99.98%

📊 All combinations processed successfully!
✅ Metrics saved to: prophet_state_all_output\prophet_all_metrics.csv
         District        Crop     MAE     RMSE    R2  MAPE(%)  Accuracy(%)
0   All Districts  full_state  148.73   199.35  0.88     7.89        92.11
1        Bagalkot       onion  378.66   550.26  0.42    36.42        63.58
2        Bagalkot       wheat  150.28   275.75  0.70     6.67        93.33
3       Bangalore    capsicum  551.75   749.05  0.48    22.02        77.98
4       Bangalore       onion  400.05   576.98  0.58    23.64        76.36
..            ...         ...     ...      ...   ...      ...          ...
85         Tumkur       wheat  196.96   471.28  0.65     6.92        93.08
86          Udupi    capsicum  495.26   734.73  0.76    14.58        85.42
87          Udupi       onion  581.35   870.86  0.55    28.82        71.18
88          Udupi      tomato  697.79  1052.64  0.35    42.36 

In [ ]:
#ARIMA

In [3]:

import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

# ==========================================================
# 📂 Paths and Setup
# ==========================================================
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "arima_state_all_output"
output_csv = os.path.join(output_folder, "arima_all_predictions")
output_graphs = os.path.join(output_folder, "arima_all_graphs")
metrics_file = os.path.join(output_folder, "arima_all_metrics.csv")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ==========================================================
# 🧮 Helper Functions
# ==========================================================
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

# ==========================================================
# 📘 Load Dataset
# ==========================================================
print(f"📘 Loading dataset: {input_file}")
df = pd.read_excel(input_file)
df.columns = [col.strip().lower() for col in df.columns]

required_cols = ['date', 'district', 'crop_type', 'average']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"❌ Missing required columns: {required_cols}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])

# ==========================================================
# 📊 Metrics Storage
# ==========================================================
metrics_list = []

# ==========================================================
# 🔁 ARIMA per (District × Crop)
# ==========================================================
groups = df.groupby(['district', 'crop_type'])

for (district, crop), group in groups:
    print(f"\n🚀 Processing: District = {district} | Crop = {crop}")

    data = group.copy().sort_values('date')
    data = data.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')

    # Fill missing values
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # ✅ Handle sparse data by using crop-level or global averages
    if data['average'].count() < 5:
        print(f"⚠️ {district} - {crop}: Not enough data, using crop-level or global average.")
        crop_level = df[df['crop_type'] == crop].groupby('date', as_index=False)['average'].mean()
        if crop_level.empty:
            print(f"⚠️ {district} - {crop}: No crop-level data found. Using overall average.")
            crop_level = df.groupby('date', as_index=False)['average'].mean()
        crop_level.set_index('date', inplace=True)
        crop_level = crop_level.asfreq('D')
        crop_level['average'] = crop_level['average'].ffill().bfill().fillna(crop_level['average'].mean())
        data = crop_level.copy()

    # ==========================================================
    # 🧩 Feature Engineering
    # ==========================================================
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(window=7).mean()
    data['MA_30'] = data['average'].rolling(window=30).mean()
    data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']] = data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].bfill().ffill()

    # ==========================================================
    # ⚙️ ARIMA Model (Simple Order)
    # ==========================================================
    y = data['average'].values
    try:
        model = ARIMA(y, order=(1,1,1))
        model_fit = model.fit()
    except Exception as e:
        print(f"❌ Skipping {district} - {crop} due to ARIMA fit error: {e}")
        continue

    # ==========================================================
    # 🔢 Forecast same length
    # ==========================================================
    predictions = model_fit.predict(start=0, end=len(y)-1)
    predictions = predictions[:len(y)]  # Ensure same length

    data['Predicted'] = predictions.round(2)

    # ==========================================================
    # 📈 Metrics Calculation
    # ==========================================================
    y_true = data['average'].values
    y_pred = data['Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'District': district,
        'Crop': crop,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # ==========================================================
    # 💾 Save Predictions
    # ==========================================================
    data.reset_index().to_csv(os.path.join(output_csv, f"{district}_{crop}_arima_daily.csv"), index=False, float_format='%.2f')

    # ==========================================================
    # 🔮 Future Forecast (30 days)
    # ==========================================================
    try:
        forecast = model_fit.forecast(steps=30)
        future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
        forecast_df = pd.DataFrame({'Date': future_dates, 'Forecast': forecast.round(2)})
        forecast_df['District'] = district
        forecast_df['Crop'] = crop
        forecast_df.to_csv(os.path.join(output_csv, f"{district}_{crop}_arima_future.csv"), index=False, float_format='%.2f')
    except Exception as e:
        print(f"⚠️ Could not generate future forecast for {district}-{crop}: {e}")

    # ==========================================================
    # 📊 Save Graph
    # ==========================================================
    plt.figure(figsize=(12,6))
    plt.plot(data.index, data['average'], label='Actual', color='blue')
    plt.plot(data.index, data['Predicted'], label='Predicted (ARIMA)', color='red', linestyle='dashed')
    if 'forecast_df' in locals():
        plt.plot(forecast_df['Date'], forecast_df['Forecast'], label='Future Forecast (30 days)', color='green', linestyle='dotted')
    plt.title(f"ARIMA Forecast - {district} | {crop}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{district}_{crop}_arima_graph.png"))
    plt.close()

    del data, y, model, model_fit
    gc.collect()

# ==========================================================
# 🧾 Save Metrics Summary
# ==========================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All combinations processed successfully!")
print(f"✅ Metrics saved to: {metrics_file}")
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx

🚀 Processing: District = All Districts | Crop = full_state
✅ All Districts - full_state | MAE=14.15, RMSE=28.28, R²=0.9975, MAPE=0.73%, Accuracy=99.27%

🚀 Processing: District = Bagalkot | Crop = onion
✅ Bagalkot - onion | MAE=13.61, RMSE=51.73, R²=0.9949, MAPE=1.06%, Accuracy=98.94%

🚀 Processing: District = Bagalkot | Crop = wheat
✅ Bagalkot - wheat | MAE=10.82, RMSE=56.97, R²=0.9873, MAPE=0.51%, Accuracy=99.49%

🚀 Processing: District = Bangalore | Crop = capsicum
✅ Bangalore - capsicum | MAE=117.4, RMSE=214.75, R²=0.9569, MAPE=4.14%, Accuracy=95.86%

🚀 Processing: District = Bangalore | Crop = onion
✅ Bangalore - onion | MAE=33.52, RMSE=93.61, R²=0.9889, MAPE=1.71%, Accuracy=98.29%

🚀 Processing: District = Bangalore | Crop = tomato
✅ Bangalore - tomato | MAE=76.43, RMSE=165.0, R²=0.9809, MAPE=4.82%, Accuracy=95.18%

🚀 Processing: District = Bangalore | Crop = wheat
✅ Bangalore - wheat | MAE=13.46, RMSE=107.32, R²=0.9651, M

In [ ]:
#SARIMAX

In [5]:

import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

# ==========================================================
# 📂 Paths and Setup
# ==========================================================
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "sarimax_state_all_output"
output_csv = os.path.join(output_folder, "sarimax_all_predictions")
output_graphs = os.path.join(output_folder, "sarimax_all_graphs")
metrics_file = os.path.join(output_folder, "sarimax_all_metrics.csv")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# ==========================================================
# 🧮 Helper Functions
# ==========================================================
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

# ==========================================================
# 📘 Load Dataset
# ==========================================================
print(f"📘 Loading dataset: {input_file}")
df = pd.read_excel(input_file)
df.columns = [col.strip().lower() for col in df.columns]

required_cols = ['date', 'district', 'crop_type', 'average']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"❌ Missing required columns: {required_cols}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])

# ==========================================================
# 📊 Metrics Storage
# ==========================================================
metrics_list = []

# ==========================================================
# 🔁 SARIMAX per (District × Crop)
# ==========================================================
groups = df.groupby(['district', 'crop_type'])

for (district, crop), group in groups:
    print(f"\n🚀 Processing: District = {district} | Crop = {crop}")

    data = group.copy().sort_values('date')
    data = data.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')

    # Fill missing values
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # ✅ Handle sparse data (less than 10 valid points)
    if data['average'].count() < 10:
        print(f"⚠️ {district} - {crop}: Not enough data, using crop-level or global average.")
        crop_level = df[df['crop_type'] == crop].groupby('date', as_index=False)['average'].mean()
        if crop_level.empty:
            print(f"⚠️ {district} - {crop}: No crop-level data found. Using overall average.")
            crop_level = df.groupby('date', as_index=False)['average'].mean()
        crop_level.set_index('date', inplace=True)
        crop_level = crop_level.asfreq('D')
        crop_level['average'] = crop_level['average'].ffill().bfill().fillna(crop_level['average'].mean())
        data = crop_level.copy()

    # ==========================================================
    # 🧩 Feature Engineering
    # ==========================================================
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(window=7).mean()
    data['MA_30'] = data['average'].rolling(window=30).mean()
    data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']] = data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].bfill().ffill()

    # Exogenous regressors
    exog = data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']]

    # ==========================================================
    # ⚙️ SARIMAX Model
    # ==========================================================
    y = data['average'].values
    try:
        model = SARIMAX(y, exog=exog, order=(1, 1, 1), seasonal_order=(0, 1, 1, 7), enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)
    except Exception as e:
        print(f"❌ Skipping {district} - {crop} due to SARIMAX fit error: {e}")
        continue

    # ==========================================================
    # 🔢 Predictions
    # ==========================================================
    predictions = model_fit.predict(start=0, end=len(y)-1, exog=exog)
    predictions = predictions[:len(y)]

    data['Predicted'] = predictions.round(2)

    # ==========================================================
    # 📈 Metrics
    # ==========================================================
    y_true = data['average'].values
    y_pred = data['Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    accuracy = round(100 - mape, 2)

    metrics_list.append({
        'District': district,
        'Crop': crop,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # ==========================================================
    # 💾 Save Predictions
    # ==========================================================
    data.reset_index().to_csv(os.path.join(output_csv, f"{district}_{crop}_sarimax_daily.csv"), index=False, float_format='%.2f')

    # ==========================================================
    # 🔮 Future Forecast (30 days)
    # ==========================================================
    try:
        future_exog = exog.tail(30).copy()
        future_exog = pd.concat([future_exog]*2, ignore_index=True).iloc[:30]  # reuse recent pattern
        forecast = model_fit.forecast(steps=30, exog=future_exog)
        future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
        forecast_df = pd.DataFrame({'Date': future_dates, 'Forecast': forecast.round(2)})
        forecast_df['District'] = district
        forecast_df['Crop'] = crop
        forecast_df.to_csv(os.path.join(output_csv, f"{district}_{crop}_sarimax_future.csv"), index=False, float_format='%.2f')
    except Exception as e:
        print(f"⚠️ Could not generate future forecast for {district}-{crop}: {e}")

    # ==========================================================
    # 📊 Plot Graph
    # ==========================================================
    plt.figure(figsize=(12,6))
    plt.plot(data.index, data['average'], label='Actual', color='blue')
    plt.plot(data.index, data['Predicted'], label='Predicted (SARIMAX)', color='red', linestyle='dashed')
    if 'forecast_df' in locals():
        plt.plot(forecast_df['Date'], forecast_df['Forecast'], label='Future Forecast (30 days)', color='green', linestyle='dotted')
    plt.title(f"SARIMAX Forecast - {district} | {crop}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{district}_{crop}_sarimax_graph.png"))
    plt.close()

    del data, y, model, model_fit, exog
    gc.collect()

# ==========================================================
# 🧾 Save Metrics
# ==========================================================
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 All combinations processed successfully!")
print(f"✅ Metrics saved to: {metrics_file}")
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx

🚀 Processing: District = All Districts | Crop = full_state
✅ All Districts - full_state | MAE=13.1, RMSE=109.58, R²=0.9627, MAPE=0.76%, Accuracy=99.24%

🚀 Processing: District = Bagalkot | Crop = onion
✅ Bagalkot - onion | MAE=15.74, RMSE=133.68, R²=0.9658, MAPE=1.2%, Accuracy=98.8%

🚀 Processing: District = Bagalkot | Crop = wheat
✅ Bagalkot - wheat | MAE=11.79, RMSE=99.16, R²=0.9615, MAPE=0.6%, Accuracy=99.4%

🚀 Processing: District = Bangalore | Crop = capsicum
✅ Bangalore - capsicum | MAE=95.55, RMSE=252.73, R²=0.9403, MAPE=3.45%, Accuracy=96.55%

🚀 Processing: District = Bangalore | Crop = onion
✅ Bangalore - onion | MAE=29.46, RMSE=115.47, R²=0.9831, MAPE=1.66%, Accuracy=98.34%

🚀 Processing: District = Bangalore | Crop = tomato
✅ Bangalore - tomato | MAE=58.93, RMSE=146.43, R²=0.985, MAPE=3.99%, Accuracy=96.01%

🚀 Processing: District = Bangalore | Crop = wheat
✅ Bangalore - wheat | MAE=18.66, RMSE=186.91, R²=0.894, MAPE

In [ ]:
#TAT+MHA

In [6]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Config
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"   # path to all-in-one excel
output_folder = "tat_mha_state_output"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
output_logs = os.path.join(output_folder, "logs")
metrics_file = os.path.join(output_folder, "tat_mha_metrics.csv")
look_back = 30
epochs = 60
batch_size = 32
future_steps = 30
d_model = 64
num_heads = 4
ff_dim = 128
dropout_rate = 0.15
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# Create folders
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_multifeature_dataset(features, target, look_back):
    # features: (N, F), target: (N,)
    X, y = [], []
    for i in range(len(features) - look_back):
        X.append(features[i:i+look_back, :])
        y.append(target[i+look_back])
    return np.array(X), np.array(y)

# -----------------------------
# TAT + MHA model builder
# -----------------------------
def build_tat_mha(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.15):
    # input_shape = (look_back, num_features)
    seq_len, num_features = input_shape

    inputs = layers.Input(shape=(seq_len, num_features), name="inputs")  # (B, T, F)
    # project features to d_model so attn & residuals align
    proj = layers.Dense(d_model, name="proj")(inputs)                     # (B, T, d_model)

    # MultiHeadAttention expects (query, value)
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads,
                                     name="mha")(proj, proj)
    attn = layers.Dropout(dropout_rate)(attn)
    x = layers.LayerNormalization(epsilon=1e-6)(attn + proj)              # residual 1

    # Feed-forward
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout_rate)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + x)                  # residual 2

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1, name="output")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="TAT_MHA")
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load dataset
# -----------------------------
print("📘 Loading dataset:", input_file)
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
required_cols = ['date','district','crop_type','average']
if not all(c in df.columns for c in required_cols):
    raise ValueError(f"Missing required columns. Required: {required_cols}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])  # remove entirely missing rows

# -----------------------------
# Prepare global crop-level series for fallback
# -----------------------------
# We'll create a crop-level series (mean across districts) aggregated by date
crop_level_df = df.groupby(['crop_type','date'], as_index=False)['average'].mean()

# -----------------------------
# Groups to process
# -----------------------------
groups = df.groupby(['district','crop_type'])

metrics_list = []

# -----------------------------
# Loop groups
# -----------------------------
for (district, crop), group in groups:
    print(f"\n🚀 Processing: {district} | {crop}")
    # copy and aggregate duplicate dates by mean
    data = group.copy().sort_values('date')
    data = data.groupby('date', as_index=False).mean(numeric_only=True)   # keep numeric cols mean
    data.set_index('date', inplace=True)
    data = data.asfreq('D')

    # fill with forward/back + mean
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # if too few points (< look_back+1), replace with crop-level time series (or global mean)
    if data['average'].count() < (look_back + 1):
        print(f"⚠️ Not enough data for {district}-{crop} (len={data['average'].count()}). Attempting crop-level fallback.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            # fallback to global average series across all crops
            print("   → No crop-level series found. Using global mean series instead.")
            global_ts = df.groupby('date', as_index=False)['average'].mean()
            crop_ts = global_ts.rename(columns={'average':'average'})
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature engineering
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(window=7).mean()
    data['MA_30'] = data['average'].rolling(window=30).mean()
    # backfill then forward fill to ensure no NaNs
    data[['Lag_1','Lag_7','MA_7','MA_30']] = data[['Lag_1','Lag_7','MA_7','MA_30']].bfill().ffill()

    # Prepare features matrix and target
    # Features order: ['average', 'Lag_1', 'Lag_7', 'MA_7', 'MA_30']  (average included so model sees target history)
    feature_cols = ['average','Lag_1','Lag_7','MA_7','MA_30']
    features_raw = data[feature_cols].values.astype('float32')   # shape (N, F)
    target_raw = data['average'].values.astype('float32')        # shape (N,)

    # Fit scalers (scale features & target separately)
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    # Fit scaler_X on entire features_raw
    scaler_X.fit(features_raw)
    scaler_y.fit(target_raw.reshape(-1,1))

    features_scaled = scaler_X.transform(features_raw)   # (N,F)
    target_scaled = scaler_y.transform(target_raw.reshape(-1,1)).flatten()

    # Create supervised sequences
    X, y = create_multifeature_dataset(features_scaled, target_scaled, look_back)
    if len(X) == 0:
        print(f"⚠️ After sequence creation, no training samples for {district}-{crop}. Skipping.")
        continue

    # Train/val split (80/20)
    n = len(X)
    split = int(n * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_val, y_val = X[split:], y[split:]

    # Build model (input_shape = (look_back, num_features))
    input_shape = (look_back, features_scaled.shape[1])
    model = build_tat_mha(input_shape, d_model=d_model, num_heads=num_heads, ff_dim=ff_dim, dropout_rate=dropout_rate)
    model.summary(print_fn=lambda s: None)  # suppress long summary

    # Callbacks
    es = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=0)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=0)
    log_path = os.path.join(output_logs, f"{district}_{crop}_tat_mha_training.log")

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[es, rl],
        verbose=1
    )

    # Save training log
    with open(log_path, "w") as f:
        f.write("Loss,Val_Loss\n")
        for i in range(len(history.history['loss'])):
            f.write(f"{history.history['loss'][i]},{history.history['val_loss'][i] if 'val_loss' in history.history else ''}\n")

    # Predict on all sequences (to align with original timeseries)
    preds_scaled = model.predict(X, batch_size=64).flatten()
    preds = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()  # real scale

    # Align predictions with dataframe: first look_back entries are NaN
    preds_full = np.concatenate([np.full(look_back, np.nan), preds])   # length = N
    data_out = data.copy().reset_index()
    data_out['Predicted'] = np.round(preds_full, 2)

    # Metrics (only where both actual & predicted exist)
    mask = ~np.isnan(data_out['Predicted'])
    if mask.sum() == 0:
        print(f"⚠️ No overlapping points for metrics for {district}-{crop}. Skipping metrics.")
        continue
    y_true = data_out.loc[mask, 'average'].values
    y_pred = data_out.loc[mask, 'Predicted'].values

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape_v = round(mape(y_true, y_pred), 2)
    accuracy = round(100 - mape_v, 2)

    metrics_list.append({
        'District': district,
        'Crop': crop,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE(%)': mape_v,
        'Accuracy(%)': accuracy
    })

    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_v}%, Acc={accuracy}%")

    # Save model
    model_path = os.path.join(output_models, f"{district}_{crop}_tat_mha.keras")
    model.save(model_path, include_optimizer=False)

    # Save historical predictions CSV
    out_csv = os.path.join(output_csv, f"{district}_{crop}_tat_mha_daily.csv")
    data_out[['date','average','Predicted']].to_csv(out_csv, index=False, float_format="%.2f")

    # -----------------------------
    # Recursive future forecast (30 days)
    # -----------------------------
    # We'll use last look_back raw features (unscaled) and iteratively generate future features
    # Start from last raw feature rows:
    raw_features = features_raw.copy()   # (N, F) unscaled features
    last_window_raw = raw_features[-look_back:].copy()   # (look_back, F)
    future_preds = []
    for step in range(future_steps):
        # Scale the window using scaler_X
        window_scaled = scaler_X.transform(last_window_raw)  # (look_back, F)
        # model expects batch dimension
        window_scaled = window_scaled[np.newaxis, ...]       # (1, look_back, F)
        pred_scaled = model.predict(window_scaled, verbose=0).flatten()[0]
        pred_unscaled = float(scaler_y.inverse_transform(np.array([[pred_scaled]]))[0,0])
        # append
        future_preds.append(round(pred_unscaled,2))

        # construct next raw feature row:
        # new_average = pred_unscaled
        prev_avgs = np.concatenate([last_window_raw[:,0], np.array([pred_unscaled])])  # previous + new
        # Lag_1 is last element before new (i.e., last_window_raw[-1,0])
        lag1 = float(last_window_raw[-1,0]) if last_window_raw.shape[0] >= 1 else pred_unscaled
        # Lag_7: take 7-th back from prev_avgs (if exists)
        lag7 = float(prev_avgs[-8]) if len(prev_avgs) >= 8 else float(prev_avgs.mean())
        ma7 = float(prev_avgs[-7:].mean()) if len(prev_avgs) >= 7 else float(prev_avgs.mean())
        # for MA30 similar
        ma30 = float(prev_avgs[-30:].mean()) if len(prev_avgs) >= 30 else float(prev_avgs.mean())

        new_row = np.array([pred_unscaled, lag1, lag7, ma7, ma30], dtype='float32')
        # append and drop oldest
        last_window_raw = np.vstack([last_window_raw[1:], new_row])

    # Save future forecast CSV
    future_dates = pd.date_range(start=data_out['date'].iat[-1] + pd.Timedelta(days=1), periods=future_steps, freq='D')
    future_df = pd.DataFrame({'date': future_dates, 'Forecast': future_preds})
    future_df['District'] = district
    future_df['Crop'] = crop
    future_df.to_csv(os.path.join(output_csv, f"{district}_{crop}_tat_mha_future.csv"), index=False, float_format="%.2f")

    # -----------------------------
    # Plot historical + pred + future
    # -----------------------------
    plt.figure(figsize=(12,6))
    plt.plot(data_out['date'], data_out['average'], label='Actual', color='blue')
    plt.plot(data_out['date'], data_out['Predicted'], label='Predicted (TAT-MHA)', color='red', linestyle='dashed')
    plt.plot(future_df['date'], future_df['Forecast'], label=f'Future {future_steps}d', color='green', linestyle='dotted')
    plt.title(f"TAT-MHA Forecast - {district} | {crop}")
    plt.xlabel("Date")
    plt.ylabel("Average Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{district}_{crop}_tat_mha_graph.png"))
    plt.close()

    # cleanup
    del model, X, y, X_train, X_val, preds_scaled, preds, data_out, future_df, last_window_raw
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All combinations processed. Metrics saved to:", metrics_file)
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx

🚀 Processing: All Districts | full_state


Epoch 1/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.0543 - mae: 0.1353 - val_loss: 0.0222 - val_mae: 0.1139 - learning_rate: 0.0010
Epoch 2/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0056 - mae: 0.0581 - val_loss: 0.0141 - val_mae: 0.0918 - learning_rate: 0.0010
Epoch 3/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0043 - mae: 0.0517 - val_loss: 0.0135 - val_mae: 0.0924 - learning_rate: 0.0010
Epoch 4/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0029 - mae: 0.0429 - val_loss: 0.0100 - val_mae: 0.0800 - learning_rate: 0.0010
Epoch 5/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0020 - mae: 0.0350 - val_loss: 0.0074 - val_mae: 0.0687 - learning_rate: 0.0010
Epoch 6/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0014 - mae: 0.0298 - val_loss: 0.0042 - val_mae: 0.0455 - learning_rate: 0.0010
Epoch 7/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0012 - mae: 0.0276 - val_loss: 0.0027 - val_mae: 0.0372 - learning_rate: 0.0010
E

Epoch 1/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.1177 - mae: 0.1621 - val_loss: 0.0015 - val_mae: 0.0283 - learning_rate: 0.0010
Epoch 2/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0064 - mae: 0.0617 - val_loss: 0.0011 - val_mae: 0.0206 - learning_rate: 0.0010
Epoch 3/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0052 - mae: 0.0555 - val_loss: 0.0026 - val_mae: 0.0461 - learning_rate: 0.0010
Epoch 4/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0044 - mae: 0.0520 - val_loss: 8.8303e-04 - val_mae: 0.0206 - learning_rate: 0.0010
Epoch 5/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0041 - mae: 0.0502 - val_loss: 0.0064 - val_mae: 0.0759 - learning_rate: 0.0010
Epoch 6/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0034 - mae: 0.0450 - val_loss: 0.0043 - val_mae: 0.0613 - learning_rate: 0.0010
Epoch 7/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0028 - mae: 0.0413 - val_loss: 7.9213e-04 - val_mae: 0.0217 - learning_rate: 

Epoch 1/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0592 - mae: 0.1373 - val_loss: 0.0182 - val_mae: 0.0796 - learning_rate: 0.0010
Epoch 2/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0039 - mae: 0.0467 - val_loss: 0.0109 - val_mae: 0.0727 - learning_rate: 0.0010
Epoch 3/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0028 - mae: 0.0383 - val_loss: 0.0090 - val_mae: 0.0570 - learning_rate: 0.0010
Epoch 4/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0024 - mae: 0.0355 - val_loss: 0.0072 - val_mae: 0.0598 - learning_rate: 0.0010
Epoch 5/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0019 - mae: 0.0315 - val_loss: 0.0074 - val_mae: 0.0500 - learning_rate: 0.0010
Epoch 6/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0018 - mae: 0.0299 - val_loss: 0.0064 - val_mae: 0.0472 - learning_rate: 0.0010
Epoch 7/60
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0015 - mae: 0.0265 - val_loss: 0.0049 - val_mae: 0.0433 - learning_rate: 0.0010


Epoch 1/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - loss: 0.0473 - mae: 0.1364 - val_loss: 0.0055 - val_mae: 0.0521 - learning_rate: 0.0010
Epoch 2/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0077 - mae: 0.0666 - val_loss: 0.0064 - val_mae: 0.0641 - learning_rate: 0.0010
Epoch 3/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0064 - mae: 0.0613 - val_loss: 0.0065 - val_mae: 0.0649 - learning_rate: 0.0010
Epoch 4/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0057 - mae: 0.0575 - val_loss: 0.0053 - val_mae: 0.0568 - learning_rate: 0.0010
Epoch 5/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0048 - mae: 0.0528 - val_loss: 0.0055 - val_mae: 0.0583 - learning_rate: 0.0010
Epoch 6/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0046 - mae: 0.0517 - val_loss: 0.0050 - val_mae: 0.0548 - learning_rate: 0.0010
Epoch 7/60
132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0042 - mae: 0.0488 - val_loss: 0.0062 - val_mae: 0.0630 - learning_rate: 0.0010
Epo

...

Epoch 1/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0863 - mae: 0.1453 - val_loss: 0.0179 - val_mae: 0.0436 - learning_rate: 0.0010
Epoch 2/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0052 - mae: 0.0573 - val_loss: 0.0161 - val_mae: 0.0391 - learning_rate: 0.0010
Epoch 3/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0042 - mae: 0.0514 - val_loss: 0.0150 - val_mae: 0.0365 - learning_rate: 0.0010
Epoch 4/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0040 - mae: 0.0503 - val_loss: 0.0155 - val_mae: 0.0456 - learning_rate: 0.0010
Epoch 5/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0030 - mae: 0.0438 - val_loss: 0.0136 - val_mae: 0.0373 - learning_rate: 0.0010
Epoch 6/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0022 - mae: 0.0368 - val_loss: 0.0134 - val_mae: 0.0361 - learning_rate: 0.0010
Epoch 7/60
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0017 - mae: 0.0329 - val_loss: 0.0131 - val_mae: 0.0321 - learning_rate: 0.0010


Epoch 1/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0453 - mae: 0.1224 - val_loss: 0.0162 - val_mae: 0.0946 - learning_rate: 0.0010
Epoch 2/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0048 - mae: 0.0524 - val_loss: 0.0121 - val_mae: 0.0802 - learning_rate: 0.0010
Epoch 3/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0038 - mae: 0.0468 - val_loss: 0.0130 - val_mae: 0.0846 - learning_rate: 0.0010
Epoch 4/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0032 - mae: 0.0419 - val_loss: 0.0126 - val_mae: 0.0820 - learning_rate: 0.0010
Epoch 5/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0028 - mae: 0.0392 - val_loss: 0.0116 - val_mae: 0.0781 - learning_rate: 0.0010
Epoch 6/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0024 - mae: 0.0364 - val_loss: 0.0115 - val_mae: 0.0771 - learning_rate: 0.0010
Epoch 7/60
124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0023 - mae: 0.0354 - val_loss: 0.0118 - val_mae: 0.0791 - learning_rate: 0.0010

Epoch 1/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0770 - mae: 0.1408 - val_loss: 0.0060 - val_mae: 0.0573 - learning_rate: 0.0010
Epoch 2/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0061 - mae: 0.0570 - val_loss: 0.0058 - val_mae: 0.0538 - learning_rate: 0.0010
Epoch 3/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0045 - mae: 0.0493 - val_loss: 0.0054 - val_mae: 0.0454 - learning_rate: 0.0010
Epoch 4/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0033 - mae: 0.0427 - val_loss: 0.0050 - val_mae: 0.0464 - learning_rate: 0.0010
Epoch 5/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0028 - mae: 0.0394 - val_loss: 0.0051 - val_mae: 0.0470 - learning_rate: 0.0010
Epoch 6/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0023 - mae: 0.0358 - val_loss: 0.0061 - val_mae: 0.0585 - learning_rate: 0.0010
Epoch 7/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0019 - mae: 0.0327 - val_loss: 0.0044 - val_mae: 0.0371 - learning_rate: 0.0010
E

Epoch 1/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0674 - mae: 0.1338 - val_loss: 0.0214 - val_mae: 0.0859 - learning_rate: 0.0010
Epoch 2/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0050 - mae: 0.0539 - val_loss: 0.0143 - val_mae: 0.0682 - learning_rate: 0.0010
Epoch 3/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0040 - mae: 0.0478 - val_loss: 0.0117 - val_mae: 0.0664 - learning_rate: 0.0010
Epoch 4/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0031 - mae: 0.0419 - val_loss: 0.0100 - val_mae: 0.0598 - learning_rate: 0.0010
Epoch 5/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.0026 - mae: 0.0380 - val_loss: 0.0114 - val_mae: 0.0659 - learning_rate: 0.0010
Epoch 6/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0022 - mae: 0.0345 - val_loss: 0.0098 - val_mae: 0.0596 - learning_rate: 0.0010
Epoch 7/60
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0019 - mae: 0.0314 - val_loss: 0.0101 - val_mae: 0.0639 - learning_rate: 0.0010
E

Epoch 1/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.2665 - mae: 0.3708 - val_loss: 0.0040 - val_mae: 0.0519 - learning_rate: 0.0010
Epoch 2/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0343 - mae: 0.1503 - val_loss: 0.0430 - val_mae: 0.2029 - learning_rate: 0.0010
Epoch 3/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0225 - mae: 0.1233 - val_loss: 0.0529 - val_mae: 0.2265 - learning_rate: 0.0010
Epoch 4/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0170 - mae: 0.1059 - val_loss: 0.0395 - val_mae: 0.1955 - learning_rate: 0.0010
Epoch 5/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0138 - mae: 0.0956 - val_loss: 0.0286 - val_mae: 0.1664 - learning_rate: 0.0010
Epoch 6/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0127 - mae: 0.0921 - val_loss: 0.0475 - val_mae: 0.2160 - learning_rate: 5.0000e-04
Epoch 7/60
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0111 - mae: 0.0842 - val_loss: 0.0403 - val_mae: 0.1990 - learning_rate: 5.0000e-04
Epoch

In [ ]:
#TAT+MQA

In [ ]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Config
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "tat_mqa_state_output_fixed2"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
metrics_file = os.path.join(output_folder, "tat_mqa_metrics.csv")

look_back = 30
future_steps = 30
epochs = 50
batch_size = 32
d_model = 64        # final internal dimension
num_heads = 4
# For multi-query we choose per-head depth; when concatenated -> d_model
depth = d_model // num_heads
ff_dim = 128
dropout_rate = 0.15
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Helpers
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_sequences(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i+look_back])
        ys.append(y[i+look_back])
    return np.array(Xs), np.array(ys)

# -----------------------------
# Robust Multi-Query Attention layer (shared K,V; separate Q per head)
# - No einsum strings, only tf.matmul + reshape/concat
# -----------------------------
class MultiQueryAttentionLayer(layers.Layer):
    def __init__(self, num_heads, depth, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        assert depth > 0, "depth must be > 0"
        self.num_heads = num_heads
        self.depth = depth
        self.dropout = layers.Dropout(dropout)
        # Q per head (each produces (B, T, depth))
        self.q_projs = [layers.Dense(depth, name=f"q_proj_{i}") for i in range(num_heads)]
        # Shared K and V (produce (B, T, depth))
        self.k_proj = layers.Dense(depth, name="k_proj_shared")
        self.v_proj = layers.Dense(depth, name="v_proj_shared")
        # Output projection to restore d_model
        self.out = layers.Dense(depth * num_heads, name="out_proj")  # will produce d_model when depth*num_heads = d_model

    def call(self, x, training=False):
        # x: (B, T, d_model) or (B,T,F) depending on upstream projection
        B = tf.shape(x)[0]
        T = tf.shape(x)[1]

        # Shared K and V -> shape (B, T, depth)
        K = self.k_proj(x)  # (B, T, depth)
        V = self.v_proj(x)  # (B, T, depth)

        # For attention matmul we want K^T per batch: we will use tf.matmul(Q_head, K_transposed)
        # K_transposed for matmul: for each batch compute (depth, T). TF batched matmul handles this.

        head_contexts = []
        for q_layer in self.q_projs:
            Q = q_layer(x)  # (B, T, depth)
            # scores = Q @ K^T  -> (B, T, T)
            # tf.matmul handles batch: tf.matmul(Q, K, transpose_b=True) => (B, T, T)
            scores = tf.matmul(Q, K, transpose_b=True)  # (B, T, T)
            scores = scores / tf.math.sqrt(tf.cast(self.depth, tf.float32))
            weights = tf.nn.softmax(scores, axis=-1)  # (B, T, T)
            weights = self.dropout(weights, training=training)
            # context = weights @ V -> (B, T, depth)
            context = tf.matmul(weights, V)  # (B, T, depth)
            head_contexts.append(context)

        # Concatenate head contexts along last axis -> (B, T, depth * num_heads) == d_model
        concat = tf.concat(head_contexts, axis=-1)  # (B, T, d_model)
        out = self.out(concat)  # (B, T, d_model) — final linear (optional)
        return out

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"num_heads": self.num_heads, "depth": self.depth})
        return cfg

# -----------------------------
# Build TAT + MQA model
# -----------------------------
def build_tat_mqa(input_shape, d_model=64, num_heads=4, depth=16, ff_dim=128, dropout=0.15):
    seq_len, num_features = input_shape
    inputs = layers.Input(shape=(seq_len, num_features), name="inputs")
    # project features to d_model
    proj = layers.Dense(d_model, name="proj")(inputs)  # (B, T, d_model)

    # Multi-Query Attention: we feed proj and let layer produce (B,T,d_model)
    attn_out = MultiQueryAttentionLayer(num_heads=num_heads, depth=depth, dropout=dropout)(proj)
    attn_out = layers.Dropout(dropout)(attn_out)
    x = layers.LayerNormalization(epsilon=1e-6)(proj + attn_out)

    # Feed-forward
    ff = layers.Dense(ff_dim, activation="relu")(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1, name="output")(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="TAT_MQA_fixed")
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load dataset & prepare
# -----------------------------
print("📘 Loading dataset:", input_file)
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
required = ['date', 'district', 'crop_type', 'average']
if not all(c in df.columns for c in required):
    raise ValueError(f"Missing required columns. Need: {required}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])

# crop-level fallback time series (mean across districts for each crop/date)
crop_level_df = df.groupby(['crop_type','date'], as_index=False)['average'].mean()

metrics = []

# -----------------------------
# Iterate over each (district, crop)
# -----------------------------
groups = df.groupby(['district','crop_type'])
for (district, crop), group in groups:
    print(f"\n🚀 Processing: {district} | {crop}")

    # aggregate duplicate dates with mean
    data = group.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)

    # ensure daily frequency
    data = data.asfreq('D')

    # fill missing average by forward/backward/mean
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # fallback to crop-level series if too few points
    if data['average'].count() < (look_back + 1):
        print(" ⚠ Sparse series; falling back to crop-level aggregated series.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            crop_ts = df.groupby('date', as_index=False)['average'].mean()
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature engineering (lags and moving averages)
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(7).mean()
    data['MA_30'] = data['average'].rolling(30).mean()
    data[['Lag_1','Lag_7','MA_7','MA_30']] = data[['Lag_1','Lag_7','MA_7','MA_30']].bfill().ffill()

    feature_cols = ['average','Lag_1','Lag_7','MA_7','MA_30']
    raw_X = data[feature_cols].values.astype('float32')
    raw_y = data['average'].values.astype('float32')

    # scaling
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(raw_X)
    y_scaled = scaler_y.fit_transform(raw_y.reshape(-1,1)).flatten()

    # create sequences
    X, y = create_sequences(X_scaled, y_scaled, look_back)
    if len(X) == 0:
        print(" ⚠ Not enough windows after sequence creation. Skipping.")
        continue

    # train/val split
    n = len(X)
    split = int(n * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_val, y_val = X[split:], y[split:]

    # build and train model
    input_shape = (look_back, X.shape[2])
    model = build_tat_mqa(input_shape, d_model=d_model, num_heads=num_heads, depth=depth, ff_dim=ff_dim, dropout=dropout_rate)

    es = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=0)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6, verbose=0)

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # predict full series
    preds_scaled = model.predict(X, batch_size=64).flatten()
    preds_inv = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
    preds_full = np.concatenate([np.full(look_back, np.nan), preds_inv])

    out_df = data.reset_index()
    out_df['Predicted'] = np.round(preds_full, 2)

    # metrics (only where prediction exists)
    mask = ~np.isnan(out_df['Predicted'])
    if mask.sum() == 0:
        print(" ⚠ No overlapping predicted points for metrics; skipping metric calculation.")
    else:
        y_true = out_df.loc[mask, 'average'].values
        y_pred = out_df.loc[mask, 'Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape_v = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        acc = round(100 - mape_v, 2)
        metrics.append({'District': district, 'Crop': crop, 'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE(%)': mape_v, 'Accuracy(%)': acc})
        print(f" ✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_v}%, Acc={acc}%")

    # save model and CSV and graph
    safe_name = f"{district}_{crop}".replace(" ", "_").replace("/", "_")
    model.save(os.path.join(output_models, f"{safe_name}_tat_mqa.keras"), include_optimizer=False)
    out_df[['date','average','Predicted']].to_csv(os.path.join(output_csv, f"{safe_name}_tat_mqa_daily.csv"), index=False, float_format="%.2f")

    # 30-day recursive forecast (uses raw_X last window)
    raw_window = raw_X[-look_back:].copy()
    future_preds = []
    for _ in range(future_steps):
        win_scaled = scaler_X.transform(raw_window)
        pred_scaled = model.predict(win_scaled[np.newaxis, ...], verbose=0).flatten()[0]
        pred_unscaled = float(scaler_y.inverse_transform([[pred_scaled]])[0,0])
        future_preds.append(round(pred_unscaled, 2))
        # build features for next step: average, lag1, lag7, ma7, ma30
        past_avgs = np.concatenate([raw_X[:,0], np.array(future_preds)])
        lag1 = raw_window[-1,0] if raw_window.shape[0] >= 1 else pred_unscaled
        lag7 = past_avgs[-8] if len(past_avgs) >= 8 else float(np.mean(past_avgs))
        ma7 = float(np.mean(past_avgs[-7:])) if len(past_avgs) >= 7 else float(np.mean(past_avgs))
        ma30 = float(np.mean(past_avgs[-30:])) if len(past_avgs) >= 30 else float(np.mean(past_avgs))
        new_row = np.array([pred_unscaled, lag1, lag7, ma7, ma30], dtype='float32')
        raw_window = np.vstack([raw_window[1:], new_row])

    future_dates = pd.date_range(start=out_df['date'].iat[-1] + pd.Timedelta(days=1), periods=future_steps, freq='D')
    future_df = pd.DataFrame({'date': future_dates, 'Forecast': future_preds})
    future_df.to_csv(os.path.join(output_csv, f"{safe_name}_tat_mqa_future.csv"), index=False, float_format="%.2f")

    # plot
    plt.figure(figsize=(12,6))
    plt.plot(out_df['date'], out_df['average'], label='Actual', color='blue')
    plt.plot(out_df['date'], out_df['Predicted'], label='Predicted (TAT-MQA)', color='red', linestyle='--')
    plt.plot(future_df['date'], future_df['Forecast'], label='Future (30d)', color='green', linestyle='dotted')
    plt.title(f"TAT-MQA Forecast - {district} | {crop}")
    plt.xlabel("Date"); plt.ylabel("Average Price")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{safe_name}_tat_mqa_graph.png"))
    plt.close()

    # cleanup
    del model, X, y, X_train, X_val, preds_scaled, preds_inv, out_df, future_df, raw_window
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All done. Metrics saved to:", metrics_file)
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx

🚀 Processing: All Districts | full_state
Epoch 1/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - loss: 0.0490 - mae: 0.1339 - val_loss: 0.0338 - val_mae: 0.1510 - learning_rate: 0.0010
Epoch 2/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0059 - mae: 0.0594 - val_loss: 0.0206 - val_mae: 0.1232 - learning_rate: 0.0010
Epoch 3/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0037 - mae: 0.0478 - val_loss: 0.0078 - val_mae: 0.0707 - learning_rate: 0.0010
Epoch 4/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0028 - mae: 0.0412 - val_loss: 0.0041 - val_mae: 0.0468 - learning_rate: 0.0010
Epoch 5/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0023 - mae: 0.0381 - val_loss: 0.0058 - val_mae: 0.0615 - learning_rate: 0.0010
Epoch 6/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0017 - mae: 0.0328 - val_loss: 0.0065 - val_mae: 0.0678 - learning_rate: 0.0010
Epoch 7/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
#TAT+GQA

In [8]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Config
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "tat_gqa_state_output"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
metrics_file = os.path.join(output_folder, "tat_gqa_metrics.csv")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

look_back = 30
future_steps = 30
epochs = 50
batch_size = 32
d_model = 64
num_heads = 4
group_size = 2   # number of heads per query group
ff_dim = 128
dropout_rate = 0.15
np.random.seed(42)
tf.random.set_seed(42)

# -----------------------------
# Helper Functions
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_dataset(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i+look_back])
        ys.append(y[i+look_back])
    return np.array(Xs), np.array(ys)

# -----------------------------
# Custom Grouped Query Attention
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, d_model, num_heads=4, group_size=2, dropout=0.15):
        super().__init__()
        assert num_heads % group_size == 0, "num_heads must be divisible by group_size"
        self.num_heads = num_heads
        self.group_size = group_size
        self.d_model = d_model
        self.depth = d_model // num_heads
        self.dropout = dropout

        self.Wq = layers.Dense(d_model)
        self.Wk = layers.Dense(d_model)
        self.Wv = layers.Dense(d_model)
        self.dense = layers.Dense(d_model)

    def call(self, v, k, q):
        B = tf.shape(q)[0]
        Q = self.Wq(q)
        K = self.Wk(k)
        V = self.Wv(v)

        Q = tf.reshape(Q, (B, -1, self.num_heads, self.depth))
        K = tf.reshape(K, (B, -1, self.num_heads, self.depth))
        V = tf.reshape(V, (B, -1, self.num_heads, self.depth))

        # Group heads together for grouped queries
        grouped_heads = self.num_heads // self.group_size
        Q_groups = tf.reshape(Q, (B, -1, grouped_heads, self.group_size * self.depth))
        K_groups = tf.reshape(K, (B, -1, grouped_heads, self.group_size * self.depth))
        V_groups = tf.reshape(V, (B, -1, grouped_heads, self.group_size * self.depth))

        attn_scores = tf.matmul(Q_groups, K_groups, transpose_b=True) / tf.math.sqrt(tf.cast(self.depth, tf.float32))
        attn_weights = tf.nn.softmax(attn_scores, axis=-1)
        attn_output = tf.matmul(attn_weights, V_groups)

        attn_output = tf.reshape(attn_output, (B, -1, self.d_model))
        output = self.dense(attn_output)
        return output

# -----------------------------
# Build TAT + GQA model
# -----------------------------
def build_tat_gqa(input_shape, d_model=64, num_heads=4, group_size=2, ff_dim=128, dropout=0.15):
    seq_len, num_features = input_shape
    inputs = layers.Input(shape=(seq_len, num_features))

    proj = layers.Dense(d_model)(inputs)
    attn_out = GroupedQueryAttention(d_model, num_heads, group_size, dropout)(proj, proj, proj)
    attn_out = layers.Dropout(dropout)(attn_out)
    x = layers.LayerNormalization(epsilon=1e-6)(proj + attn_out)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    ff = layers.Dropout(dropout)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs, name="TAT_GQA")
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load Dataset
# -----------------------------
print(f"📘 Loading dataset: {input_file}")
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
required = ['date', 'district', 'crop_type', 'average']
if not all(c in df.columns for c in required):
    raise ValueError(f"Missing required columns: {required}")

df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date')
df = df.dropna(subset=['average'])

# crop-level fallback
crop_level_df = df.groupby(['crop_type', 'date'], as_index=False)['average'].mean()

metrics_list = []

# -----------------------------
# Process each (district, crop)
# -----------------------------
for (district, crop), group in df.groupby(['district', 'crop_type']):
    print(f"\n🚀 Processing: {district} | {crop}")

    data = group.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    if len(data) < look_back + 1:
        print(f"⚠️ Not enough points for {district}-{crop}. Using crop-level fallback.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            crop_ts = df.groupby('date', as_index=False)['average'].mean()
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature Engineering
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(7).mean()
    data['MA_30'] = data['average'].rolling(30).mean()
    data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']] = data[['Lag_1', 'Lag_7', 'MA_7', 'MA_30']].bfill().ffill()

    feature_cols = ['average', 'Lag_1', 'Lag_7', 'MA_7', 'MA_30']
    X_raw = data[feature_cols].values
    y_raw = data['average'].values

    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X_raw)
    y_scaled = scaler_y.fit_transform(y_raw.reshape(-1, 1)).flatten()

    X, y = create_dataset(X_scaled, y_scaled, look_back)
    if len(X) == 0:
        print(f"⚠️ No data window for {district}-{crop}. Skipping.")
        continue

    n = len(X)
    split = int(n * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_val, y_val = X[split:], y[split:]

    # Build Model
    model = build_tat_gqa((look_back, X.shape[2]), d_model, num_heads, group_size, ff_dim, dropout_rate)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=0)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predictions
    preds_scaled = model.predict(X).flatten()
    preds = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
    preds_full = np.concatenate([np.full(look_back, np.nan), preds])

    data_out = data.copy().reset_index()
    data_out['Predicted'] = np.round(preds_full, 2)

    # Metrics
    mask = ~np.isnan(data_out['Predicted'])
    y_true, y_pred = data_out.loc[mask, 'average'], data_out.loc[mask, 'Predicted']

    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape_val = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape_val, 2)
    metrics_list.append({'District': district, 'Crop': crop,
                         'MAE': mae, 'RMSE': rmse, 'R2': r2,
                         'MAPE(%)': mape_val, 'Accuracy(%)': acc})
    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_val}%, Acc={acc}%")

    # Save predictions
    data_out[['date', 'average', 'Predicted']].to_csv(
        os.path.join(output_csv, f"{district}_{crop}_tat_gqa_daily.csv"), index=False, float_format="%.2f"
    )
    model.save(os.path.join(output_models, f"{district}_{crop}_tat_gqa.keras"), include_optimizer=False)

    # 30-day Future Forecast
    last_window = X_scaled[-look_back:]
    raw_last = X_raw[-look_back:]
    future_preds = []
    for _ in range(future_steps):
        pred_scaled = model.predict(last_window[np.newaxis, ...], verbose=0).flatten()[0]
        pred_unscaled = scaler_y.inverse_transform([[pred_scaled]])[0,0]
        future_preds.append(round(pred_unscaled, 2))
        new_row = np.array([pred_unscaled,
                            raw_last[-1,0], raw_last[-7,0] if len(raw_last)>=7 else pred_unscaled,
                            np.mean(raw_last[-7:,0]) if len(raw_last)>=7 else pred_unscaled,
                            np.mean(raw_last[-30:,0]) if len(raw_last)>=30 else pred_unscaled])
        raw_last = np.vstack([raw_last[1:], new_row])
        last_window = scaler_X.transform(raw_last)

    future_dates = pd.date_range(start=data_out['date'].iloc[-1] + pd.Timedelta(days=1),
                                 periods=future_steps, freq='D')
    future_df = pd.DataFrame({'Date': future_dates, 'Forecast': future_preds})
    future_df.to_csv(os.path.join(output_csv, f"{district}_{crop}_tat_gqa_future.csv"),
                     index=False, float_format="%.2f")

    # Plot
    plt.figure(figsize=(12,6))
    plt.plot(data_out['date'], data_out['average'], label='Actual', color='blue')
    plt.plot(data_out['date'], data_out['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='--')
    plt.plot(future_df['Date'], future_df['Forecast'], label='Future (30d)', color='green', linestyle='dotted')
    plt.title(f"TAT-GQA Forecast - {district} | {crop}")
    plt.xlabel("Date"); plt.ylabel("Average Price")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{district}_{crop}_tat_gqa_graph.png"))
    plt.close()

    del model, X, y, X_train, X_val
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics_list).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All combinations done. Metrics saved to:", metrics_file)
print(metrics_df)

129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 5.3034e-04 - mae: 0.0117 - val_loss: 0.0081 - val_mae: 0.0478 - learning_rate: 3.1250e-05
Epoch 44/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.3605e-04 - mae: 0.0120 - val_loss: 0.0081 - val_mae: 0.0496 - learning_rate: 3.1250e-05
Epoch 45/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.4229e-04 - mae: 0.0119 - val_loss: 0.0079 - val_mae: 0.0459 - learning_rate: 3.1250e-05
Epoch 46/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.3526e-04 - mae: 0.0117 - val_loss: 0.0079 - val_mae: 0.0476 - learning_rate: 1.5625e-05
Epoch 47/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 5.2739e-04 - mae: 0.0118 - val_loss: 0.0079 - val_mae: 0.0481 - learning_rate: 1.5625e-05
Epoch 48/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.3902e-04 - mae: 0.0119 - val_loss: 0.0079 - val_mae: 0.0503 - learning_rate: 1.5625e-05
Epoch 49/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 5.1966e-04 - mae: 0.0116 - val_loss: 0

In [ ]:
#TAT+HA

In [ ]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Configuration
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "tat_ha_state_output"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
metrics_file = os.path.join(output_folder, "tat_ha_metrics.csv")

look_back = 30
epochs = 50
batch_size = 32
d_model = 64
num_heads = 4
ff_dim = 128
dropout_rate = 0.15
future_steps = 30

np.random.seed(42)
tf.random.set_seed(42)

for d in [output_folder, output_models, output_csv, output_graphs]:
    os.makedirs(d, exist_ok=True)

# -----------------------------
# Helper functions
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_sequences(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i+look_back])
        ys.append(y[i+look_back])
    return np.array(Xs), np.array(ys)

# -----------------------------
# ✅ Hierarchical Attention Block
# -----------------------------
class HierarchicalAttention(layers.Layer):
    """
    Two-level attention:
    - Feature-level attention: learns which features are most important per timestep.
    - Temporal-level attention: learns which timesteps are most important overall.
    """
    def __init__(self, d_model, num_heads, dropout_rate=0.1, ff_dim=128):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.feature_dense = layers.Dense(d_model, activation='tanh')
        self.feature_att = layers.Dense(1, activation=None)

        self.temporal_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=self.depth)
        self.dropout = layers.Dropout(dropout_rate)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(d_model)
        ])
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        # x shape: (batch, time, features)
        # Feature-level attention
        feat_proj = self.feature_dense(x)  # (B, T, d_model)
        feat_scores = self.feature_att(feat_proj)  # (B, T, 1)
        feat_weights = tf.nn.softmax(feat_scores, axis=2)
        feat_context = tf.reduce_sum(feat_proj * feat_weights, axis=2, keepdims=True)

        # Temporal-level attention
        temp_out = self.temporal_att(feat_context, feat_context)
        temp_out = self.dropout(temp_out, training=training)
        out1 = self.norm1(feat_context + temp_out)

        # Feed-forward
        ff_out = self.ffn(out1)
        ff_out = self.dropout(ff_out, training=training)
        out2 = self.norm2(out1 + ff_out)
        return out2

# -----------------------------
# Build Model
# -----------------------------
def build_tat_ha(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout=0.15):
    seq_len, num_features = input_shape
    inputs = layers.Input(shape=(seq_len, num_features))
    x = layers.Dense(d_model)(inputs)

    # Hierarchical Attention block
    x = HierarchicalAttention(d_model, num_heads, dropout, ff_dim)(x)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs, outputs, name="TAT_HA")
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load dataset
# -----------------------------
print("📘 Loading dataset:", input_file)
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date').dropna(subset=['average'])

crop_level_df = df.groupby(['crop_type', 'date'], as_index=False)['average'].mean()

metrics = []

# -----------------------------
# Training loop per district-crop
# -----------------------------
for (district, crop), group in df.groupby(['district', 'crop_type']):
    print(f"\n🚀 Processing: {district} | {crop}")

    data = group.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    if len(data) < look_back + 1:
        print("⚠️ Sparse data — using crop-level fallback.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            crop_ts = df.groupby('date', as_index=False)['average'].mean()
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature engineering
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(7).mean()
    data['MA_30'] = data['average'].rolling(30).mean()
    data[['Lag_1','Lag_7','MA_7','MA_30']] = data[['Lag_1','Lag_7','MA_7','MA_30']].bfill().ffill()

    feature_cols = ['average','Lag_1','Lag_7','MA_7','MA_30']
    raw_X, raw_y = data[feature_cols].values, data['average'].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(raw_X)
    y_scaled = scaler_y.fit_transform(raw_y.reshape(-1,1)).flatten()

    X, y = create_sequences(X_scaled, y_scaled, look_back)
    if len(X) == 0:
        print("⚠️ Insufficient data — skipping.")
        continue

    n = len(X)
    split = int(n * 0.8)
    X_train, y_train, X_val, y_val = X[:split], y[:split], X[split:], y[split:]

    model = build_tat_ha((look_back, X.shape[2]), d_model, num_heads, ff_dim, dropout_rate)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    preds_scaled = model.predict(X).flatten()
    preds = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
    preds_full = np.concatenate([np.full(look_back, np.nan), preds])
    out_df = data.reset_index()
    out_df['Predicted'] = np.round(preds_full, 2)

    mask = ~np.isnan(out_df['Predicted'])
    y_true, y_pred = out_df.loc[mask,'average'], out_df.loc[mask,'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape_v = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape_v, 2)

    metrics.append({'District':district,'Crop':crop,'MAE':mae,'RMSE':rmse,'R2':r2,'MAPE(%)':mape_v,'Accuracy(%)':acc})
    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_v}%, Acc={acc}%")

    # Save model, CSV, and graph
    safe_name = f"{district}_{crop}".replace(" ", "_").replace("/", "_")
    model.save(os.path.join(output_models, f"{safe_name}_tat_ha.keras"), include_optimizer=False)
    out_df[['date','average','Predicted']].to_csv(os.path.join(output_csv, f"{safe_name}_tat_ha_daily.csv"), index=False, float_format="%.2f")

    # Plot graph
    plt.figure(figsize=(12,6))
    plt.plot(out_df['date'], out_df['average'], label='Actual', color='blue')
    plt.plot(out_df['date'], out_df['Predicted'], label='Predicted', color='red', linestyle='--')
    plt.title(f"TAT+HA Forecast: {district} | {crop}")
    plt.xlabel("Date"); plt.ylabel("Average Price")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{safe_name}_tat_ha_graph.png"))
    plt.close()

    del model, X, y
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All done. Metrics saved to:", metrics_file)
print(metrics_df)

📘 Loading dataset: dataset/Crop_price_forecast_Daily.xlsx


In [ ]:
#LSTM

In [ ]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Configuration
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "lstm_state_output"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
metrics_file = os.path.join(output_folder, "lstm_metrics.csv")

look_back = 30
epochs = 50
batch_size = 32
future_steps = 30
lstm_units = 128
dropout_rate = 0.2

np.random.seed(42)
tf.random.set_seed(42)

for d in [output_folder, output_models, output_csv, output_graphs]:
    os.makedirs(d, exist_ok=True)

# -----------------------------
# Helper Functions
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_sequences(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i+look_back])
        ys.append(y[i+look_back])
    return np.array(Xs), np.array(ys)

# -----------------------------
# LSTM Model Builder
# -----------------------------
def build_lstm(input_shape, units=128, dropout=0.2):
    model = models.Sequential([
        layers.LSTM(units, return_sequences=True, input_shape=input_shape),
        layers.Dropout(dropout),
        layers.LSTM(units // 2),
        layers.Dropout(dropout),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load Dataset
# -----------------------------
print("📘 Loading dataset:", input_file)
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date').dropna(subset=['average'])

crop_level_df = df.groupby(['crop_type', 'date'], as_index=False)['average'].mean()

metrics = []

# -----------------------------
# Process per (District, Crop)
# -----------------------------
for (district, crop), group in df.groupby(['district', 'crop_type']):
    print(f"\n🚀 Processing: {district} | {crop}")

    data = group.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # Fallback for sparse data
    if len(data) < look_back + 1:
        print("⚠️ Sparse data — using crop-level fallback.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            crop_ts = df.groupby('date', as_index=False)['average'].mean()
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature Engineering
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(7).mean()
    data['MA_30'] = data['average'].rolling(30).mean()
    data[['Lag_1','Lag_7','MA_7','MA_30']] = data[['Lag_1','Lag_7','MA_7','MA_30']].bfill().ffill()

    feature_cols = ['average','Lag_1','Lag_7','MA_7','MA_30']
    raw_X, raw_y = data[feature_cols].values, data['average'].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(raw_X)
    y_scaled = scaler_y.fit_transform(raw_y.reshape(-1,1)).flatten()

    X, y = create_sequences(X_scaled, y_scaled, look_back)
    if len(X) == 0:
        print("⚠️ Insufficient data — skipping.")
        continue

    n = len(X)
    split = int(n * 0.8)
    X_train, y_train, X_val, y_val = X[:split], y[:split], X[split:], y[split:]

    # Build model
    model = build_lstm((look_back, X.shape[2]), units=lstm_units, dropout=dropout_rate)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train model
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predictions
    preds_scaled = model.predict(X, batch_size=64).flatten()
    preds = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
    preds_full = np.concatenate([np.full(look_back, np.nan), preds])
    out_df = data.reset_index()
    out_df['Predicted'] = np.round(preds_full, 2)

    # Metrics
    mask = ~np.isnan(out_df['Predicted'])
    y_true, y_pred = out_df.loc[mask,'average'], out_df.loc[mask,'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape_v = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape_v, 2)
    metrics.append({'District':district,'Crop':crop,'MAE':mae,'RMSE':rmse,'R2':r2,'MAPE(%)':mape_v,'Accuracy(%)':acc})
    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_v}%, Acc={acc}%")

    # Save model and results
    safe_name = f"{district}_{crop}".replace(" ", "_").replace("/", "_")
    model.save(os.path.join(output_models, f"{safe_name}_lstm.keras"), include_optimizer=False)
    out_df[['date','average','Predicted']].to_csv(os.path.join(output_csv, f"{safe_name}_lstm_daily.csv"), index=False, float_format="%.2f")

    # Future 30-day forecast
    raw_window = raw_X[-look_back:].copy()
    future_preds = []
    for _ in range(future_steps):
        scaled_window = scaler_X.transform(raw_window)
        pred_scaled = model.predict(scaled_window[np.newaxis, ...], verbose=0).flatten()[0]
        pred_unscaled = float(scaler_y.inverse_transform([[pred_scaled]])[0,0])
        future_preds.append(round(pred_unscaled, 2))

        # Generate next step’s features
        lag1 = raw_window[-1,0] if len(raw_window) >= 1 else pred_unscaled
        lag7 = raw_window[-7,0] if len(raw_window) >= 7 else np.mean(raw_window[:,0])
        ma7 = np.mean(np.append(raw_window[:,0], pred_unscaled)[-7:])
        ma30 = np.mean(np.append(raw_window[:,0], pred_unscaled)[-30:])
        new_row = np.array([pred_unscaled, lag1, lag7, ma7, ma30], dtype='float32')
        raw_window = np.vstack([raw_window[1:], new_row])

    future_dates = pd.date_range(start=out_df['date'].iloc[-1] + pd.Timedelta(days=1),
                                 periods=future_steps, freq='D')
    future_df = pd.DataFrame({'date':future_dates, 'Forecast':future_preds})
    future_df.to_csv(os.path.join(output_csv, f"{safe_name}_lstm_future.csv"), index=False, float_format="%.2f")

    # Plot graph
    plt.figure(figsize=(12,6))
    plt.plot(out_df['date'], out_df['average'], label='Actual', color='blue')
    plt.plot(out_df['date'], out_df['Predicted'], label='Predicted (LSTM)', color='red', linestyle='--')
    plt.plot(future_df['date'], future_df['Forecast'], label='Future (30 days)', color='green', linestyle='dotted')
    plt.title(f"LSTM Forecast: {district} | {crop}")
    plt.xlabel("Date"); plt.ylabel("Average Price")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{safe_name}_lstm_graph.png"))
    plt.close()

    del model, X, y
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All done. Metrics saved to:", metrics_file)
print(metrics_df)

In [ ]:
#GRU

In [ ]:

import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# -----------------------------
# Configuration
# -----------------------------
input_file = "dataset/Crop_price_forecast_Daily.xlsx"
output_folder = "gru_state_output"
output_models = os.path.join(output_folder, "models")
output_csv = os.path.join(output_folder, "predictions")
output_graphs = os.path.join(output_folder, "graphs")
metrics_file = os.path.join(output_folder, "gru_metrics.csv")

look_back = 30
epochs = 50
batch_size = 32
future_steps = 30
gru_units = 128
dropout_rate = 0.2

np.random.seed(42)
tf.random.set_seed(42)

for d in [output_folder, output_models, output_csv, output_graphs]:
    os.makedirs(d, exist_ok=True)

# -----------------------------
# Helper Functions
# -----------------------------
def parse_dates_safe(series):
    try:
        return pd.to_datetime(series, dayfirst=True)
    except:
        return pd.to_datetime(series, dayfirst=False)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan

def create_sequences(X, y, look_back):
    Xs, ys = [], []
    for i in range(len(X) - look_back):
        Xs.append(X[i:i+look_back])
        ys.append(y[i+look_back])
    return np.array(Xs), np.array(ys)

# -----------------------------
# GRU Model Builder
# -----------------------------
def build_gru(input_shape, units=128, dropout=0.2):
    model = models.Sequential([
        layers.GRU(units, return_sequences=True, input_shape=input_shape),
        layers.Dropout(dropout),
        layers.GRU(units // 2),
        layers.Dropout(dropout),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Load Dataset
# -----------------------------
print("📘 Loading dataset:", input_file)
df = pd.read_excel(input_file)
df.columns = [c.strip().lower() for c in df.columns]
df['date'] = parse_dates_safe(df['date'])
df = df.sort_values('date').dropna(subset=['average'])

# crop-level fallback if needed
crop_level_df = df.groupby(['crop_type', 'date'], as_index=False)['average'].mean()

metrics = []

# -----------------------------
# Process per (District, Crop)
# -----------------------------
for (district, crop), group in df.groupby(['district', 'crop_type']):
    print(f"\n🚀 Processing: {district} | {crop}")

    # Prepare data for each (district, crop)
    data = group.groupby('date', as_index=False).mean(numeric_only=True)
    data.set_index('date', inplace=True)
    data = data.asfreq('D')
    data['average'] = data['average'].ffill().bfill().fillna(data['average'].mean())

    # Fallback if data is too short
    if len(data) < look_back + 1:
        print("⚠️ Sparse data — using crop-level fallback.")
        crop_ts = crop_level_df[crop_level_df['crop_type'] == crop].copy()
        if crop_ts.empty:
            crop_ts = df.groupby('date', as_index=False)['average'].mean()
        crop_ts.set_index('date', inplace=True)
        crop_ts = crop_ts.asfreq('D')
        crop_ts['average'] = crop_ts['average'].ffill().bfill().fillna(crop_ts['average'].mean())
        data = crop_ts.copy()

    # Feature Engineering
    data['Lag_1'] = data['average'].shift(1)
    data['Lag_7'] = data['average'].shift(7)
    data['MA_7'] = data['average'].rolling(7).mean()
    data['MA_30'] = data['average'].rolling(30).mean()
    data[['Lag_1','Lag_7','MA_7','MA_30']] = data[['Lag_1','Lag_7','MA_7','MA_30']].bfill().ffill()

    feature_cols = ['average','Lag_1','Lag_7','MA_7','MA_30']
    raw_X, raw_y = data[feature_cols].values, data['average'].values

    scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
    X_scaled = scaler_X.fit_transform(raw_X)
    y_scaled = scaler_y.fit_transform(raw_y.reshape(-1,1)).flatten()

    X, y = create_sequences(X_scaled, y_scaled, look_back)
    if len(X) == 0:
        print("⚠️ Insufficient data — skipping.")
        continue

    n = len(X)
    split = int(n * 0.8)
    X_train, y_train, X_val, y_val = X[:split], y[:split], X[split:], y[split:]

    # Build GRU model
    model = build_gru((look_back, X.shape[2]), units=gru_units, dropout=dropout_rate)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    rl = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Train
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=epochs, batch_size=batch_size, callbacks=[es, rl], verbose=1)

    # Predict
    preds_scaled = model.predict(X, batch_size=64).flatten()
    preds = scaler_y.inverse_transform(preds_scaled.reshape(-1,1)).flatten()
    preds_full = np.concatenate([np.full(look_back, np.nan), preds])
    out_df = data.reset_index()
    out_df['Predicted'] = np.round(preds_full, 2)

    # Metrics
    mask = ~np.isnan(out_df['Predicted'])
    y_true, y_pred = out_df.loc[mask,'average'], out_df.loc[mask,'Predicted']
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    mape_v = round(mean_absolute_percentage_error(y_true, y_pred), 2)
    acc = round(100 - mape_v, 2)
    metrics.append({'District':district,'Crop':crop,'MAE':mae,'RMSE':rmse,'R2':r2,'MAPE(%)':mape_v,'Accuracy(%)':acc})
    print(f"✅ {district} - {crop} | MAE={mae}, RMSE={rmse}, R²={r2}, MAPE={mape_v}%, Acc={acc}%")

    # Save model and results
    safe_name = f"{district}_{crop}".replace(" ", "_").replace("/", "_")
    model.save(os.path.join(output_models, f"{safe_name}_gru.keras"), include_optimizer=False)
    out_df[['date','average','Predicted']].to_csv(os.path.join(output_csv, f"{safe_name}_gru_daily.csv"), index=False, float_format="%.2f")

    # Future 30-day forecast
    raw_window = raw_X[-look_back:].copy()
    future_preds = []
    for _ in range(future_steps):
        scaled_window = scaler_X.transform(raw_window)
        pred_scaled = model.predict(scaled_window[np.newaxis, ...], verbose=0).flatten()[0]
        pred_unscaled = float(scaler_y.inverse_transform([[pred_scaled]])[0,0])
        future_preds.append(round(pred_unscaled, 2))

        # Build next features
        lag1 = raw_window[-1,0] if len(raw_window) >= 1 else pred_unscaled
        lag7 = raw_window[-7,0] if len(raw_window) >= 7 else np.mean(raw_window[:,0])
        ma7 = np.mean(np.append(raw_window[:,0], pred_unscaled)[-7:])
        ma30 = np.mean(np.append(raw_window[:,0], pred_unscaled)[-30:])
        new_row = np.array([pred_unscaled, lag1, lag7, ma7, ma30], dtype='float32')
        raw_window = np.vstack([raw_window[1:], new_row])

    future_dates = pd.date_range(start=out_df['date'].iloc[-1] + pd.Timedelta(days=1),
                                 periods=future_steps, freq='D')
    future_df = pd.DataFrame({'date':future_dates, 'Forecast':future_preds})
    future_df.to_csv(os.path.join(output_csv, f"{safe_name}_gru_future.csv"), index=False, float_format="%.2f")

    # Plot graph
    plt.figure(figsize=(12,6))
    plt.plot(out_df['date'], out_df['average'], label='Actual', color='blue')
    plt.plot(out_df['date'], out_df['Predicted'], label='Predicted (GRU)', color='red', linestyle='--')
    plt.plot(future_df['date'], future_df['Forecast'], label='Future (30 days)', color='green', linestyle='dotted')
    plt.title(f"GRU Forecast: {district} | {crop}")
    plt.xlabel("Date"); plt.ylabel("Average Price")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(os.path.join(output_graphs, f"{safe_name}_gru_graph.png"))
    plt.close()

    del model, X, y
    gc.collect()

# Save metrics
metrics_df = pd.DataFrame(metrics).round(2)
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 All done. Metrics saved to:", metrics_file)
print(metrics_df)